In [1]:
# 1609492611000, 1641028611000, 1672564611000 (2021, 2022, 2023) 1677632461000, 1654045200000 (2022,6,1), 1659315600000 (2022,8,1), 1650045261000

In [2]:
# ! conda install -c conda-forge ta --yes

In [3]:
import requests
import pandas as pd
import pandas_ta as p_ta
import ta
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

In [4]:
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover, resample_apply, cross

C:\Users\George\anaconda3\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [5]:
interval_arr = ['15m', '1h', '4h', '1d']
start_time_arr = [1641028611000]

In [6]:
timezone = 8
symbol = 'ethusdt'
symbol_C = symbol.upper()
interval = '15m'

start_time = 1641028611000
# end_time = 1659315600000
end_time = round(time.time() * 1000)

# step between timestamps in milliseconds
step = 60000 * 3600

In [7]:
def create_raw(symbol, interval_arr, start_time, end_time, step):
    
    url = "https://api.binance.com/api/v3/klines"
    
    for interval in interval_arr:

        raw_df = pd.DataFrame()
        
        for timestamp in range(start_time, end_time, step):
            params = {"symbol": symbol_C,
                      "interval": interval,
                      "startTime": timestamp,
                      "endTime": timestamp + step}
            response = requests.get(url, params=params).json()
            out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                                   "Volume", "Close_Time", "Quote asset volume",
                                                   "Number of trades", "Taker buy base asset volume",
                                                   "Taker buy quote asset volume", "Ignore"])
            raw_df = pd.concat([raw_df, out], axis = 0)

        raw_df = raw_df[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]

        raw_df.to_hdf(f'klines_{symbol}_{interval}.h5', key='df', mode='w')
        print(f"Created {symbol}_{interval}")

In [8]:
loop_start_time = time.time()
create_raw(symbol, interval_arr, min(start_time_arr), end_time, step)
loop_end_time = time.time()
print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")

Created ethusdt_15m
Created ethusdt_1h
Created ethusdt_4h
Created ethusdt_1d
Time taken to execute for loop: 85.86460304260254 seconds


In [9]:
h5 = pd.read_hdf(f'klines_{symbol}_{interval_arr[0]}.h5', key='df')
print(h5.tail(1))
print(len(h5))

        Close_Time           Open          Close           High  \
115  1681525799999  2083.73000000  2084.46000000  2084.48000000   

               Low        Volume  
115  2083.26000000  324.69020000  
44991


In [10]:
def time_format(timezone):
#     df['Close_Time'] = pd.to_datetime(df['Close_Time'], unit='ms') + pd.Timedelta(hours=timezone)
#     df['Close_Time'] = df['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=timezone)
#     df.index = df.index.strftime('%Y-%m-%d %H:%M:%S')

In [11]:
klines_cache = {}

def get_klines(symbol, interval, start_time, end_time):
    if (symbol, interval) not in klines_cache:
        klines_cache[(symbol, interval)] = pd.read_hdf(f'klines_{symbol}_{interval}.h5', key='df')

    df = klines_cache[(symbol, interval)].query(f"Close_Time >= {start_time} and Close_Time <= {end_time}")

    df = df[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']].astype(float)
    df = df.set_index('Close_Time')
#     df = df.reset_index(drop=True)

    df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=timezone)
    df = df[~df.index.duplicated(keep='first')]
#     df.index = df.index.strftime('%Y-%m-%d %H:%M:%S')
#     df.index = pd.to_datetime(df.index)
#     df = df[df.Volume!=0]
#     df = df[0:2000]
    print(df)
    df.to_csv('df.csv')

    return df

In [12]:
cash = 2500
leverage = 10
commission = 0.05/100

In [14]:
import multiprocessing as mp

if __name__ == '__main__':
    mp.set_start_method('spawn')
    
# spawn

In [16]:
class RsiOscillator(Strategy):
    
    rsi_up = 70
    rsi_low = 30
    rsi_window = 14
    
    kd_int = 14
    d_int = 3
    
    ema1 = 8
    ema2 = 18
    ema3 = 38

    
    # Do as much initial computation as possible
    def init(self):
        
        close_series = pd.Series(self.data.Close, index = self.data.index)
        high_series = pd.Series(self.data.High, index = self.data.index)
        low_series = pd.Series(self.data.Low, index = self.data.index)
            
        self.rsi_15m = self.I(p_ta.rsi, close_series, self.rsi_window)
        self.rsi_1h = resample_apply('H', p_ta.rsi, close_series, self.rsi_window)
#         self.rsi_4h = resample_apply('4H', p_ta.rsi, close_series, self.rsi_window)
        
#         self.ema1_15m = self.I(ta.trend.ema_indicator, close_series, window=self.ema1)  
#         self.ema2_15m = self.I(ta.trend.ema_indicator, close_series, window=self.ema2)   
#         self.ema3_15m = self.I(ta.trend.ema_indicator, close_series, window=self.ema3)   

#         self.ema1_1h = resample_apply('H', ta.trend.ema_indicator, close_series, window=self.ema1)  
#         self.ema2_1h = resample_apply('H', ta.trend.ema_indicator, close_series, window=self.ema2)   
#         self.ema3_1h = resample_apply('H', ta.trend.ema_indicator, close_series, window=self.ema3) 

        self.ema1_4h = resample_apply('4H', ta.trend.ema_indicator, close_series, window=self.ema1)  
        self.ema2_4h = resample_apply('4H', ta.trend.ema_indicator, close_series, window=self.ema2)   
        self.ema3_4h = resample_apply('4H', ta.trend.ema_indicator, close_series, window=self.ema3) 
        
        kd = ta.momentum.StochasticOscillator(high = high_series,
                                              low = low_series,
                                              close = close_series,
                                              window = self.kd_int,
                                              smooth_window = self.d_int)
        
        self.k_15m = self.I(kd.stoch)
        self.d_15m = self.I(kd.stoch_signal)
        
        self.atr_15m = self.I(ta.volatility.average_true_range, high_series, low_series, close_series)


    # Step through bars one by one

    def next(self):
        
        low_series = pd.Series(self.data.Low, index = self.data.index)[-1]

        if (self.rsi_1h > self.rsi_up):
            self.position.close()
        elif (crossover(self.k_15m, self.d_15m) and
              (self.rsi_1h[-10:] < self.rsi_low).any() and
              self.ema1_4h > self.ema3_4h):
                self.buy(size = 0.075, sl = low_series - 2 * self.atr_15m)

bt = Backtest(get_klines(symbol, interval, start_time, end_time),
             RsiOscillator,
             cash = cash,
             commission = commission,
             margin = 1/leverage)

# stats = bt.run()
stats = bt.optimize(
        rsi_up = range(60, 85, 5),
        rsi_low = range(15, 40, 5),
        rsi_window = range(10, 30, 2),
        maximize = 'Equity Final [$]',
        return_heatmap=True,
        return_optimization=True,
        method = 'skopt')

print(stats)
bt.plot(filename = 'backtesting.html')

                                  Open    Close     High      Low     Volume
Close_Time                                                                  
2022-01-01 17:44:59.999000064  3722.52  3713.95  3726.91  3709.38  1052.7796
2022-01-01 17:59:59.999000064  3713.95  3718.67  3720.15  3710.87   811.7021
2022-01-01 18:14:59.999000064  3718.67  3719.64  3731.88  3717.97  1415.8797
2022-01-01 18:29:59.999000064  3719.64  3691.22  3719.90  3688.18  3344.1421
2022-01-01 18:44:59.999000064  3691.22  3698.61  3699.97  3673.46  3851.5917
...                                ...      ...      ...      ...        ...
2023-04-15 09:14:59.999000064  2086.01  2082.88  2087.41  2080.01  4930.2012
2023-04-15 09:29:59.999000064  2082.89  2076.98  2085.00  2071.13  5940.3720
2023-04-15 09:44:59.999000064  2076.98  2083.26  2083.62  2076.97  2759.5858
2023-04-15 09:59:59.999000064  2083.27  2081.26  2084.88  2081.26  2415.5924
2023-04-15 10:14:59.999000064  2081.26  2083.73  2090.67  2081.26  3460.2271

C:\Users\George\AppData\Local\Temp\ipykernel_6436\1078588476.py:63: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(get_klines(symbol, interval, start_time, end_time),


Backtest.optimize:   0%|          | 0/200 [00:00<?, ?it/s]

(Start                     2022-01-01 17:44...
End                       2023-04-15 10:14...
Duration                    468 days 16:30:00
Exposure Time [%]                   21.209158
Equity Final [$]                  5508.223953
Equity Peak [$]                   5546.463953
Return [%]                         120.328958
Buy & Hold Return [%]              -43.894506
Return (Ann.) [%]                   84.683429
Volatility (Ann.) [%]               97.346655
Sharpe Ratio                         0.869916
Sortino Ratio                        2.919054
Calmar Ratio                         2.236842
Max. Drawdown [%]                  -37.858475
Avg. Drawdown [%]                   -2.979368
Max. Drawdown Duration      187 days 22:45:00
Avg. Drawdown Duration        5 days 00:18:00
# Trades                                   98
Win Rate [%]                        29.591837
Best Trade [%]                      18.553566
Worst Trade [%]                     -3.311157
Avg. Trade [%]                   

C:\Users\George\anaconda3\lib\site-packages\backtesting\_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '2H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


ValueError: Length of values (2) does not match length of index (1)

In [ ]:
from backtesting.test import GOOG
print(pd.Series(GOOG.Close))

In [ ]:
# class RsiOscillator(Strategy):
    
    rsi_up = 70
    rsi_low = 30
    rsi_window = 14
    
    kd_int = 14
    d_int = 3
    
    ema1 = 8
    ema2 = 18
    ema3 = 38

    # Do as much initial computation as possible
    def init(self):
        
        close_series = pd.Series(self.data.Close, index = self.data.index)
        high_series = pd.Series(self.data.High, index = self.data.index)
        low_series = pd.Series(self.data.Low, index = self.data.index)
        print(close_series)
            
        self.rsi_15m = self.I(p_ta.rsi, close_series, self.rsi_window)
        self.rsi_1h = resample_apply("W-FRI", p_ta.rsi, close_series, self.rsi_window)
        
        self.ema1 = self.I(ta.trend.ema_indicator, close_series, window=self.ema1)  
        self.ema2 = self.I(ta.trend.ema_indicator, close_series, window=self.ema2)   
        self.ema3 = self.I(ta.trend.ema_indicator, close_series, window=self.ema3)   

        
        kd = ta.momentum.StochasticOscillator(high = high_series,
                                              low = low_series,
                                              close = close_series,
                                              window = self.kd_int,
                                              smooth_window = self.d_int)
        
        self.k = self.I(kd.stoch)
        self.d = self.I(kd.stoch_signal)

    # Step through bars one by one

    def next(self):
        if (self.rsi_15m[-5:] > self.rsi_up).any() and crossover(self.d, self.k):
            self.position.close()
        elif (self.rsi_15m[-5:] < self.rsi_low).any() and crossover(self.k, self.d):
            self.buy(size = 0.1)

bt = Backtest(GOOG,
             RsiOscillator,
             cash = cash,
             commission = commission,
             margin = 1/leverage)

stats = bt.run()
print(stats)
bt.plot(filename = 'backtesting.html')